<a href="https://colab.research.google.com/github/simplifiedcomputing/weed_detection/blob/main/weed_detection_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weed Detection Inference
Locate medical plants for human and animal health using object detection and classification.

## Install

- FastAI (training loop library)
- IceVision (computer vision framework)
- MMDetection and Yolo v5 (neural net models)
- Exif (gps image extraction)
- Simplekml (google map overlay)

In [ ]:
!pip install openmim -q
!mim install mmcv-full
!mim install mmdet

!pip install git+git://github.com/airctic/icevision.git#egg=icevision[all] -U -q
!pip install git+git://github.com/airctic/icedata.git -U -q
!pip install yolov5-icevision -U -q

!pip install exif -q
!pip install Simplekml -q

In [1]:
# Restart kernel after installation
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'restart': True, 'status': 'ok'}

## Imports


In [3]:
from icevision.all import * # process object detection
from icevision.models.checkpoint import * # import trained models
from google.colab import files # up/download files
import pandas as pd # Image metadata processing
pd.set_option('display.precision', 10)
import requests # download files from url
from exif import Image # gps image extraction
import imageio # split images into smaller pieces
import glob # get local filenames
import simplekml # create map files

#Loading model
Because we have saved our model weights with the model metadata, we only need to call model_from_checkpoint(checkpoint_path): No other arguments (model_name, backbone_name, classes, img_size) are needed. All the information is already embedded in the checkpoint file.

In [4]:
checkpoint_path = 'https://simplified-computing.de/weed_detection_split_checkpoint.pth'
checkpoint_and_model = model_from_checkpoint(checkpoint_path)

load checkpoint from http path: https://simplified-computing.de/weed_detection_split_checkpoint.pth


Downloading: "https://simplified-computing.de/weed_detection_split_checkpoint.pth" to /root/.cache/torch/hub/checkpoints/weed_detection_split_checkpoint.pth


  0%|          | 0.00/27.0M [00:00<?, ?B/s]

In [6]:
# Just logging the info
model_type = checkpoint_and_model["model_type"]
backbone = checkpoint_and_model["backbone"]
class_map = checkpoint_and_model["class_map"]
img_size = checkpoint_and_model["img_size"]
model_type, backbone, class_map, img_size

(<module 'icevision.models.ultralytics.yolov5' from '/usr/local/lib/python3.7/dist-packages/icevision/models/ultralytics/yolov5/__init__.py'>,
 <ClassMap: {'background': 0, 'herbzeitlose': 1}>,
 1920)

In [7]:
model = checkpoint_and_model["model"]

##Transforms

In [8]:
img_size = checkpoint_and_model["img_size"]
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(img_size), tfms.A.Normalize()])

#Data

###Load images

Multi image upload

In [ ]:
uploaded = files.upload()
filenames_uploads = list(uploaded.keys())

Saving weed_detection_test_DJI_0252.jpg to weed_detection_test_DJI_0252.jpg


or use this testimage

In [9]:
!wget https://simplified-computing.de/weed_detection_DJI_0252.JPG
filenames_uploads = glob.glob('*.JPG')

--2021-11-14 14:52:31--  https://simplified-computing.de/weed_detection_DJI_0252.JPG
Resolving simplified-computing.de (simplified-computing.de)... 35.198.83.164
Connecting to simplified-computing.de (simplified-computing.de)|35.198.83.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11045731 (11M) [image/jpeg]
Saving to: ‘weed_detection_DJI_0252.JPG’

weed_detection_DJI_ 100%[===================>]  10.53M  13.1MB/s    in 0.8s    

2021-11-14 14:52:33 (13.1 MB/s) - ‘weed_detection_DJI_0252.JPG’ saved [11045731/11045731]



or use the trainingset

In [ ]:
!wget https://simplified-computing.de/weed_detection_data.zip
!unzip weed_detection_data.zip
filenames_uploads = glob.glob('*.JPG')

--2021-11-04 13:24:29--  https://simplified-computing.de/weed_detection_data.zip
Resolving simplified-computing.de (simplified-computing.de)... 35.198.83.164
Connecting to simplified-computing.de (simplified-computing.de)|35.198.83.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 350677090 (334M) [application/zip]
Saving to: ‘weed_detection_data.zip’

weed_detection_data 100%[===================>] 334.43M  26.4MB/s    in 15s     

2021-11-04 13:24:44 (22.6 MB/s) - ‘weed_detection_data.zip’ saved [350677090/350677090]

Archive:  weed_detection_data.zip
  inflating: DJI_0252.JPG            
  inflating: DJI_0253.JPG            
  inflating: DJI_0260.JPG            
  inflating: DJI_0263.JPG            
  inflating: DJI_0264.JPG            
  inflating: DJI_0266.JPG            
  inflating: DJI_0272.JPG            
  inflating: DJI_0273.JPG            
  inflating: DJI_0274.JPG            
  inflating: DJI_0275.JPG            
  inflating: DJI_0277.JPG        

#### Extract GPS coordinates

In [10]:
def decimal_coords(coords, ref):
 decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
 if ref == "S" or ref == "W":
     decimal_degrees = -decimal_degrees
 return decimal_degrees

def lon(img_file):
 lon_coord = decimal_coords(Image(img_file).gps_longitude, Image(img_file).gps_longitude_ref)
 return lon_coord

def lat(img_file):
 lat_coord = decimal_coords(Image(img_file).gps_latitude, Image(img_file).gps_latitude_ref)
 return lat_coord

lon_coords = list(map(lon, filenames_uploads))
lat_coords = list(map(lat, filenames_uploads))

#### Split images in smaller pieces

In [11]:
def make_splits(images):

    os.mkdir('split_image_dir')
    save_path = 'split_image_dir'
    
    splits_w = [[.0, .5], [.5, 1.]]
    splits_h = [[.0, .5], [.5, 1.]]    
    
    for num, image in enumerate(images):
        im = imageio.imread(image)
        h, w = im.shape[:2]
        
        name = image.split('/')[-1]
        for s_h in splits_h:
            for s_w in splits_w:
                img = im[int(h*s_h[0]):int(h*s_h[1]), int(w*s_w[0]):int(w*s_w[1]), :]
                new_w = int(s_w[0]*w)
                new_h = int(s_h[0]*h)
                
                save_name = os.path.join(save_path, f'{num}_{new_h}_{new_w}___{name}')
                imageio.imwrite(save_name, img)

make_splits(filenames_uploads)

In [12]:
# list of split files
os.listdir('split_image_dir')

['0_1500_0___weed_detection_DJI_0252.JPG',
 '0_1500_2000___weed_detection_DJI_0252.JPG',
 '0_0_2000___weed_detection_DJI_0252.JPG',
 '0_0_0___weed_detection_DJI_0252.JPG']

In [ ]:
# show sample image
img = PIL.Image.open('split_image_dir/0_0_0___weed_detection_test_DJI_0252.jpg')
img

###Image metadata processing
filenames, gps, bboxes

In [13]:
# get filename_split
filenames_split = glob.glob('split_image_dir/*')

# extract filename from filename_split to hava a connection to merge with coords
filenames_split_df = pd.DataFrame(
    {'filename': filenames_split,
     'filenames_split': filenames_split
    })
filenames_split_df['filename'] = filenames_split_df.filenames_split.str.extract('___(.*)')     
filenames_split_df

,filename,filenames_split
0,weed_detection_DJI_0252.JPG,split_image_dir/0_1500_0___weed_detection_DJI_0252.JPG
1,weed_detection_DJI_0252.JPG,split_image_dir/0_1500_2000___weed_detection_DJI_0252.JPG
2,weed_detection_DJI_0252.JPG,split_image_dir/0_0_2000___weed_detection_DJI_0252.JPG
3,weed_detection_DJI_0252.JPG,split_image_dir/0_0_0___weed_detection_DJI_0252.JPG


In [14]:
# merge filenames with coords
filenames_uploads_df = pd.DataFrame(
    {'filename': filenames_uploads,
     'lon': lon_coords,
     'lat': lat_coords
    })

filenames_uploads_df

,filename,lon,lat
0,weed_detection_DJI_0252.JPG,10.8517537819,49.8584265391


In [15]:
# merge filenames_split, filenames, coords
filenames_coords_df = pd.merge(filenames_split_df, filenames_uploads_df, how="left", on=["filename"])
filenames_coords_df

,filename,filenames_split,lon,lat
0,weed_detection_DJI_0252.JPG,split_image_dir/0_1500_0___weed_detection_DJI_0252.JPG,10.8517537819,49.8584265391
1,weed_detection_DJI_0252.JPG,split_image_dir/0_1500_2000___weed_detection_DJI_0252.JPG,10.8517537819,49.8584265391
2,weed_detection_DJI_0252.JPG,split_image_dir/0_0_2000___weed_detection_DJI_0252.JPG,10.8517537819,49.8584265391
3,weed_detection_DJI_0252.JPG,split_image_dir/0_0_0___weed_detection_DJI_0252.JPG,10.8517537819,49.8584265391


# Create dataset

In [16]:
filenames_split = glob.glob('split_image_dir/*')
images_array = [PIL.Image.open(Path(name)) for name in filenames_split]
infer_ds = Dataset.from_images(images_array, valid_tfms, class_map=class_map)

#Predict - All at once

Simply call predict:

In [ ]:
preds = model_type.predict(model, infer_ds, keep_images=True, detection_threshold=0.01)

#Predict - In batches
If the memory is not enough to predict everything at once, break it down into smaller batches with infer_dataloader:

In [17]:
infer_dl = model_type.infer_dl(infer_ds, batch_size=1, shuffle=False)
preds = model_type.predict_from_dl(model=model, infer_dl=infer_dl, keep_images=True, detection_threshold=0.01)

  0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


#Visualize

Let´s take a look at the bboxes

In [18]:
preds[0].pred.detection.bboxes

[<BBox (xmin:1489.20849609375, ymin:752.24853515625, xmax:1513.52099609375, ymax:779.3721923828125)>,
 <BBox (xmin:1297.5462646484375, ymin:529.5830078125, xmax:1315.5692138671875, ymax:548.708984375)>,
 <BBox (xmin:1298.7578125, ymin:523.59375, xmax:1325.060302734375, ymax:547.3790283203125)>]

For visualizing the predictions we can use show_preds:

In [ ]:
show_preds(preds=preds)

## Localization

Calculate GPS coordinates for detected objects.

In [19]:
# grab bboxes
bbox_list = []
for i in range(len(preds)):
    for bbox in preds[i].pred.detection.bboxes:
      x = i, bbox.xmin, bbox.xmax, bbox.ymin, bbox.ymax
      bbox_list.append(x)
bbox_df = pd.DataFrame(bbox_list)
bbox_df.columns = ['record', 'bbox_xmin', 'bbox_xmax', 'bbox_ymin', 'bbox_ymax']

In [20]:
# add index column for merge
filenames_coords_df = filenames_coords_df.reset_index().rename({'index':'record'}, axis = 'columns')

In [21]:
# merge bboxes with filenames and coords
filenames_coords_bboxes_df = pd.merge(filenames_coords_df, bbox_df, how="right", on=["record"])
filenames_coords_bboxes_df

,record,filename,filenames_split,lon,lat,bbox_xmin,bbox_xmax,bbox_ymin,bbox_ymax
0,0,weed_detection_DJI_0252.JPG,split_image_dir/0_1500_0___weed_detection_DJI_0252.JPG,10.8517537819,49.8584265391,1489.2084960938,1513.5209960938,752.2485351562,779.3721923828
1,0,weed_detection_DJI_0252.JPG,split_image_dir/0_1500_0___weed_detection_DJI_0252.JPG,10.8517537819,49.8584265391,1297.5462646484,1315.5692138672,529.5830078125,548.7089843750
2,0,weed_detection_DJI_0252.JPG,split_image_dir/0_1500_0___weed_detection_DJI_0252.JPG,10.8517537819,49.8584265391,1298.7578125000,1325.0603027344,523.5937500000,547.3790283203
3,2,weed_detection_DJI_0252.JPG,split_image_dir/0_0_2000___weed_detection_DJI_0252.JPG,10.8517537819,49.8584265391,1036.7962646484,1075.8653564453,1323.0286865234,1354.0303955078
4,2,weed_detection_DJI_0252.JPG,split_image_dir/0_0_2000___weed_detection_DJI_0252.JPG,10.8517537819,49.8584265391,391.8636169434,415.5443420410,1440.9694824219,1463.4897460938
5,3,weed_detection_DJI_0252.JPG,split_image_dir/0_0_0___weed_detection_DJI_0252.JPG,10.8517537819,49.8584265391,1334.0032958984,1365.3548583984,1575.4737548828,1624.3929443359
6,3,weed_detection_DJI_0252.JPG,split_image_dir/0_0_0___weed_detection_DJI_0252.JPG,10.8517537819,49.8584265391,1383.4060058594,1409.4050292969,1541.1202392578,1573.9913330078
7,3,weed_detection_DJI_0252.JPG,split_image_dir/0_0_0___weed_detection_DJI_0252.JPG,10.8517537819,49.8584265391,1331.7768554688,1354.4123535156,1558.2354736328,1600.8631591797


In [22]:
# bbox center in pixel
filenames_coords_bboxes_df['bbox_xmean'] = filenames_coords_bboxes_df[['bbox_xmin', 'bbox_xmax']].mean(axis=1)
filenames_coords_bboxes_df['bbox_ymean'] = filenames_coords_bboxes_df[['bbox_ymin', 'bbox_ymax']].mean(axis=1)


In [23]:
# image dimensions in cm
image_x_cm = 710
image_y_cm = 550
splitimage_x_cm = image_x_cm / 2
splitimage_y_cm = image_y_cm / 2
splitimage_x_cm, splitimage_y_cm

(355.0, 275.0)

In [24]:
# image dimensions in pixel
image_x_resolution_px = 4000
image_y_resolution_px = 3000
splitimage_x_resolution_px = image_x_resolution_px / 2
splitimage_y_resolution_px = image_y_resolution_px / 2
splitimage_x_resolution_scaled_px = 1920
splitimage_y_resolution_scaled_px = splitimage_y_resolution_px * splitimage_x_resolution_scaled_px / splitimage_x_resolution_px
splitimage_x_resolution_scaled_px, splitimage_y_resolution_scaled_px


(1920, 1440.0)

In [25]:
# convert units: amount of pixels per cm
pixel_per_cm_x = splitimage_x_resolution_scaled_px / splitimage_x_cm
pixel_per_cm_y = splitimage_y_resolution_scaled_px / splitimage_y_cm
pixel_per_cm_x, pixel_per_cm_y

(5.408450704225352, 5.236363636363636)

In [27]:
# convert units: amount of degree per cm (in germany)
# using http://seanavigation.narod.ru/progi/degree.html

# lon (aka längengrad, x, width)
degree_per_m_lon = 71906
cm_per_degree_lon = 1 / degree_per_m_lon / 100

# lat (aka breitengrad ,y , height)
degree_per_m_lat = 111226
cm_per_degree_lat = 1 / degree_per_m_lat / 100

cm_per_degree_lon, cm_per_degree_lat

(1.3907045309153617e-07, 8.990703612464712e-08)

In [28]:
# convert units: amount of degree per pixel
pixel_per_degree = pixel_per_cm_x / cm_per_degree_lon
pixel_per_degree


38890005.633802816

In [29]:
# bbox coords
filenames_coords_bboxes_df['bbox_lon'] = filenames_coords_bboxes_df['lon'] + (filenames_coords_bboxes_df['bbox_xmean'] / pixel_per_degree)
filenames_coords_bboxes_df['bbox_lat'] = filenames_coords_bboxes_df['lat'] - (filenames_coords_bboxes_df['bbox_ymean'] / pixel_per_degree)
filenames_coords_bboxes_df[['lon', 'lat', 'bbox_xmean', 'bbox_ymean', 'bbox_lon', 'bbox_lat']]

,lon,lat,bbox_xmean,bbox_ymean,bbox_lon,bbox_lat
0,10.8517537819,49.8584265391,1501.3647460938,765.8103637695,10.8517923873,49.8584068474
1,10.8517537819,49.8584265391,1306.5577392578,539.1459960938,10.8517873781,49.8584126757
2,10.8517537819,49.8584265391,1311.9090576172,535.4863891602,10.8517875157,49.8584127698
3,10.8517537819,49.8584265391,1056.3308105469,1338.5295410156,10.8517809439,49.8583921207
4,10.8517537819,49.8584265391,403.7039794922,1452.2296142578,10.8517641625,49.8583891971
5,10.8517537819,49.8584265391,1349.6790771484,1599.9333496094,10.8517884869,49.8583853991
6,10.8517537819,49.8584265391,1396.4055175781,1557.5557861328,10.8517896884,49.8583864888
7,10.8517537819,49.8584265391,1343.0946044922,1579.5493164062,10.8517883176,49.8583859233


In [30]:
# coords adjustment for splitfiles
filenames_coords_bboxes_df['adjustment_lon_splitfile_top_left'] = filenames_coords_bboxes_df[filenames_coords_bboxes_df['filenames_split'].str.contains('0_0_0___')]['bbox_lon'] - (splitimage_x_resolution_scaled_px / pixel_per_degree)
filenames_coords_bboxes_df['adjustment_lat_splitfile_top_left'] = filenames_coords_bboxes_df[filenames_coords_bboxes_df['filenames_split'].str.contains('0_0_0___')]['bbox_lat'] + (splitimage_y_resolution_scaled_px / pixel_per_degree)
filenames_coords_bboxes_df['adjustment_lon_splitfile_top_right'] = filenames_coords_bboxes_df[filenames_coords_bboxes_df['filenames_split'].str.contains('0_2000___')]['bbox_lon']
filenames_coords_bboxes_df['adjustment_lat_splitfile_top_right'] = filenames_coords_bboxes_df[filenames_coords_bboxes_df['filenames_split'].str.contains('0_2000___')]['bbox_lat'] + (splitimage_y_resolution_scaled_px / pixel_per_degree)
filenames_coords_bboxes_df['adjustment_lon_splitfile_bottom_left'] = filenames_coords_bboxes_df[filenames_coords_bboxes_df['filenames_split'].str.contains('1500_0___')]['bbox_lon'] + (splitimage_x_resolution_scaled_px / pixel_per_degree)
filenames_coords_bboxes_df['adjustment_lat_splitfile_bottom_left'] = filenames_coords_bboxes_df[filenames_coords_bboxes_df['filenames_split'].str.contains('1500_0___')]['bbox_lat']
filenames_coords_bboxes_df['adjustment_lon_splitfile_bottom_right'] = filenames_coords_bboxes_df[filenames_coords_bboxes_df['filenames_split'].str.contains('0_1500_2000___')]['bbox_lon']
filenames_coords_bboxes_df['adjustment_lat_splitfile_bottom_right'] = filenames_coords_bboxes_df[filenames_coords_bboxes_df['filenames_split'].str.contains('0_1500_2000___')]['bbox_lat']

# combine adjustments
filenames_coords_bboxes_df['bbox_lon_adjusted'] = filenames_coords_bboxes_df['adjustment_lon_splitfile_top_left'].combine_first(filenames_coords_bboxes_df['adjustment_lon_splitfile_bottom_left']).combine_first(filenames_coords_bboxes_df['adjustment_lon_splitfile_top_right']).combine_first(filenames_coords_bboxes_df['adjustment_lon_splitfile_bottom_right'])
filenames_coords_bboxes_df['bbox_lat_adjusted'] = filenames_coords_bboxes_df['adjustment_lat_splitfile_top_left'].combine_first(filenames_coords_bboxes_df['adjustment_lat_splitfile_top_right']).combine_first(filenames_coords_bboxes_df['adjustment_lat_splitfile_bottom_left']).combine_first(filenames_coords_bboxes_df['adjustment_lat_splitfile_bottom_right'])

filenames_coords_bboxes_df[['filenames_split', 'bbox_lon', 'bbox_lat', 'adjustment_lon_splitfile_top_left', 'adjustment_lat_splitfile_top_left', 'adjustment_lat_splitfile_top_right', 'adjustment_lon_splitfile_bottom_left', 'bbox_lon_adjusted', 'bbox_lat_adjusted']]

# Btw:
# if lon higher than marker moves to east/right
# if lat higher than marker moves to north/top

,filenames_split,bbox_lon,bbox_lat,adjustment_lon_splitfile_top_left,adjustment_lat_splitfile_top_left,adjustment_lat_splitfile_top_right,adjustment_lon_splitfile_bottom_left,bbox_lon_adjusted,bbox_lat_adjusted
0,split_image_dir/0_1500_0___weed_detection_DJI_0252.JPG,10.8517923873,49.8584068474,NaN,NaN,NaN,10.8518417573,10.8518417573,49.8584068474
1,split_image_dir/0_1500_0___weed_detection_DJI_0252.JPG,10.8517873781,49.8584126757,NaN,NaN,NaN,10.8518367481,10.8518367481,49.8584126757
2,split_image_dir/0_1500_0___weed_detection_DJI_0252.JPG,10.8517875157,49.8584127698,NaN,NaN,NaN,10.8518368857,10.8518368857,49.8584127698
3,split_image_dir/0_0_2000___weed_detection_DJI_0252.JPG,10.8517809439,49.8583921207,NaN,NaN,49.8584291482,NaN,10.8517809439,49.8584291482
4,split_image_dir/0_0_2000___weed_detection_DJI_0252.JPG,10.8517641625,49.8583891971,NaN,NaN,49.8584262246,NaN,10.8517641625,49.8584262246
5,split_image_dir/0_0_0___weed_detection_DJI_0252.JPG,10.8517884869,49.8583853991,10.8517391169,49.8584224266,NaN,NaN,10.8517391169,49.8584224266
6,split_image_dir/0_0_0___weed_detection_DJI_0252.JPG,10.8517896884,49.8583864888,10.8517403184,49.8584235163,NaN,NaN,10.8517403184,49.8584235163
7,split_image_dir/0_0_0___weed_detection_DJI_0252.JPG,10.8517883176,49.8583859233,10.8517389476,49.8584229508,NaN,NaN,10.8517389476,49.8584229508


## Map

Create Map with detected plants

In [31]:
kml = simplekml.Kml()
kml.document.name = "Herbstzeitlose"
for index, row in filenames_coords_bboxes_df.iterrows():
    bboxlon = (row['bbox_lon_adjusted'])
    bboxlat = (row['bbox_lat_adjusted'])
    kml.newpoint(name="Herbstzeitlose", coords=[(bboxlon,bboxlat)])
kml.save('herbstzeitlose.kml')
files.download('herbstzeitlose.kml')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Import herbstzeitlose.kml in google maps.

https://www.google.de/intl/de/maps/about/mymaps/

:-)